In [ ]:
%cd /Users/trancatkhanh/Downloads/GAME
from base.Splendor_v3.env import *


# Homework 3:
**1 agent tổng quán sử dụng bộ feature và bias có tỷ lệ thắng trên 60%**

# Bước 1 :Lấy bias của các action

In [ ]:
from base.Splendor_v3.env import *
import numpy as np
bias1 = [np.random.rand(getActionSize()),np.zeros(getActionSize())]
@njit()
def p_bias(state,per):
    actions = getValidActions(state)
    actions *= per[0]
    action = np.argmax(actions)
    if getReward(state) == 1:
        per[1] += per[0]
    if getReward(state) == 0:
        per[0] = np.random.rand(getActionSize())
    return action,per

win, bias = numba_main_2(p_bias, 10000,bias1,0)
# Bias tối ưu 
bias_greedy = (bias[1]/np.max(bias[1]))
print(bias_greedy)
# Check tỷ lệ thắng của bias tối ưu 
@njit()
def p_bias_optim(state,per):
    actions = getValidActions(state)
    actions *= bias_greedy
    action = np.argmax(actions)
    return action,per
win, bias = numba_main_2(p_bias_optim, 1000,bias1,0)
print(win)

# Bước 2: Ghép các feature với nhau 

In [ ]:
def get_feature5(i):
  #State old là state Hiện tại 
  #State new là state Mới 
  old = np.load('old'+f'{i}'+'.npy')
  new = np.load('new'+f'{i}'+'.npy')
  rate = 0 
  while rate<0.2 :

    # state mà có cả Value của old and new (1,0)
    o = (new>old)
    o= 1*o
    i = np.zeros((old.shape[0],1))
    for u in range(old.shape[0]) :
        if sum(o[u])>0 :
            # print(o[u])
            i[u] = 1
    #  a matrix with random value
    l = np.random.uniform(-1,1,(161,5))
    for j in range(getStateSize()):
      for vl in range(0,5):
        if l[j][vl] > np.random.randn(1):
            l[j][vl] = 1
        elif l[j][vl] < np.random.randn(1):
            l[j][vl] =  0
        else:
            l[j][vl] = -1
    #state mà kích hoạt (1,0)
    
    k = 1*((new@l)>=0)
    print(k)
    # state mà kích hoạt và có reward (1,0)
    y = np.zeros((old.shape[0],1))
    for p in range(old.shape[0]):
        if sum(k[p]) >= 1 and sum(i[p])>= 1:
            y[p] = 1
        else:
            y[p] = 0
      
    rate = (np.sum(y)/np.sum(k))
    print(rate)

  return l.reshape(5,161)
matrix_feature_check_win_rate = []
for k in range(0,1): 
  matrix  = get_feature5(k)
  matrix_feature = []
  matrix_feature.append(matrix)
  perx=[np.zeros((len(matrix_feature),getStateSize())),np.zeros((getActionSize(),(len(matrix_feature))*5))]
  for i in range(0,len(matrix_feature)):
      for j in range(getStateSize()):
          perx[0][i][j] += matrix_feature[i][0][j]
  @njit
  def p0(state,per):
      actions = getValidActions(state)
      matran1 = per[0][0]@state    
      matran1 = int(matran1 >= 0)
      actions = np.where(actions == 1)[0]
      if matran1 == 1 and k in actions :
          action = k
      else:
          actions = np.where(actions == 1)[0]
          action = actions[np.random.randint(len(actions))]
      return action,per
  win, x = numba_main_2(p0, 1000,perx,0)
  print('actiom'+f'{k}',win)
  while  win<310 :
    matrix  = get_feature5(k)
    matrix_feature = []
    matrix_feature.append(matrix)
    perx=[np.zeros((len(matrix_feature),getStateSize())),np.zeros((getActionSize(),(len(matrix_feature))*5))]
    for i in range(0,len(matrix_feature)):
        for j in range(getStateSize()):
            perx[0][i][j] += matrix_feature[i][0][j]
    win, x = numba_main_2(p0, 1000,perx,0)
    print('actiom'+f'{k}',win)

  matrix_feature_check_win_rate.append(matrix)



In [ ]:
perx=[np.zeros((75,getStateSize())),np.zeros((getActionSize(),75)]
for i in range(0,75):
    for j in range(getStateSize()):
        if perx[0][i][j] > np.random.randn(1):
            perx[0][i][j] = 1
        elif perx[0][i][j] < np.random.randn(1):
            perx[0][i][j] =  0
        else:
            perx[0][i][j] = -1
from base.Splendor_v3.env import *
@njit
def p0(state,per):
    actions = getValidActions(state)
    matran1 = per[0]@state
    for i in range(len(matran1)):
        matran1[i] = int(matran1[i] >= 0)
    matran2 = per[1]@matran1
    matran2 *= actions
    if len(np.where(matran2 == matran2[np.argmax(matran2)])[0])>1:
        matran2 += bias_greedy
        # print(matran2)
        action = np.argmax(matran2)
    else:
        action = np.argmax(matran2)
    return action,per
win, x = numba_main_2(p0, 10000,perx,0)

In [ ]:
from base.Splendor_v3.env import *
for i in range(10):
    import numpy as np
    from numba import njit 
    #Khởi tạo weight 
    perx = [np.random.randn(5,getStateSize()),np.zeros((1,5))]
    for i in range(0,5):
        for j in range(getStateSize()):
            if perx[0][i][j] > np.random.randn(1):
                perx[0][i][j] = 1
            elif perx[0][i][j] < np.random.randn(1):
                perx[0][i][j] =  0
            else:
                perx[0][i][j] = -1
    perx[1][0:6]=1

    @njit
    def p0(state,per):
        actions = getValidActions(state)
        matran1 = per[0]@state
        for i in range(len(matran1)):
            matran1[i] = int(matran1[i] >= 0)
        matran2 = per[1]@matran1
        matran2 *= actions
        if matran2 == 5 :
            action = 1 
        else:
            actions = np.where(actions == 1)[0]
            action = actions[np.random.randint(len(actions))]
            # while action == 1:
            #     action = actions[np.random.randint(len(actions))]
        return action,per
        
    win, x = numba_main_2(p0, 10000,perx,0)
    print('tỷ lệ thắng',(win/10000)*100)

9379
15
